In [43]:
import os
import re
import glob
import pickle
import numpy as np
import pandas as pd
import pandas.io.common
warnings.filterwarnings("ignore")

In [44]:
#### File Path and Variables
loc_path = r'/Users/raj2.gaurav/Desktop/' 
path = loc_path + r'/Causal Model/01. Raw Data/01. Anomaly Report/'
pickle_path = loc_path + 'Causal Model/02. Processed Data/01. Anomaly Data/'
### Input for the Circle 
Circle= 'bihar'
monthname='Nov'
year = '2022'

In [55]:
## Pre Processing
def remove_whitespaces(df):
    df_obj = df.select_dtypes(['object'])
    df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
    df.columns = df.columns.str.strip()
    return df

def drop_duplicate_records(df):
    df.drop_duplicates()
    return df

#### Collating Files 


allFiles = glob.glob(path + "/*.xlsx")
lst=[]
df_master=pd.DataFrame()
for i in allFiles:
    df=pd.read_excel(i)
    #print(df.shape)
    df_master=df_master.append(df)
    #print(df_master.shape)

/var/folders/3p/7q9_hy6x4q58xrz02_n9tvv0fryx5_/T/ipykernel_18435/26641898.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_master=df_master.append(df)
/var/folders/3p/7q9_hy6x4q58xrz02_n9tvv0fryx5_/T/ipykernel_18435/26641898.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_master=df_master.append(df)
/var/folders/3p/7q9_hy6x4q58xrz02_n9tvv0fryx5_/T/ipykernel_18435/26641898.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_master=df_master.append(df)
/var/folders/3p/7q9_hy6x4q58xrz02_n9tvv0fryx5_/T/ipykernel_18435/26641898.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_master=df_master.append(df)
/var/folders

In [56]:
df_master.head()

,Circle,City,Component,Date,Hour,Total Customers,Anomaly,Ratio Data,Ratio Voice,Ratio HSI,...,num_sectors,Indoor Frqeqency % 850,Indoor Frqeqency % 1800,Indoor Frqeqency % 2300,prb_10,prb_10_25,prb_25_70,prb_70_90,prb_90,Consistency
0,mumbai,mumbai,1099,2022-11-06 00:00:00,7,15211,HSI,1,2,6,...,27,27,25,46,20.0,48.0,20.0,0.0,2.0,NaN
1,mumbai,mumbai,1099,2022-11-06 00:00:00,6,12283,HSI,1,2,13,...,27,24,23,51,51.0,33.0,4.0,0.0,2.0,NaN
2,mumbai,mumbai,1090,2022-11-06 00:00:00,14,42770,HSI,2,3,4,...,30,20,29,50,4.0,11.0,43.0,18.0,24.0,1.0
3,mumbai,mumbai,1090,2022-11-06 00:00:00,7,25173,Voice,2,3,3,...,30,20,25,53,23.0,40.0,32.0,5.0,0.0,2.0
4,mumbai,mumbai,1090,2022-11-06 00:00:00,13,44454,HSI,2,3,4,...,30,20,29,50,3.0,14.0,43.0,18.0,22.0,NaN


### Pre Process

In [57]:
print(df_master.shape)
print(df_master.columns)

(222556, 38)
Index(['Circle', 'City', 'Component', 'Date', 'Hour', 'Total Customers',
       'Anomaly', 'Ratio Data', 'Ratio Voice', 'Ratio HSI', 'cluster_issue',
       'Data Volume', 'Data Volume Formula', 'Data Volume mean',
       'Data Volume std', 'Voice Affected Users',
       'Voice Affected Users Formula', 'Voice Affected Users mean',
       'Voice Affected Users std', 'HSI Affected Users',
       'HSI Affected Users Formula', 'HSI Affected Users mean',
       'HSI Affected Users std', 'Data % Change', 'Voice % Change',
       'HSI % Change', 'num_grids', 'num_cells', 'num_sectors',
       'Indoor Frqeqency % 850', 'Indoor Frqeqency % 1800',
       'Indoor Frqeqency % 2300', 'prb_10', 'prb_10_25', 'prb_25_70',
       'prb_70_90', 'prb_90', 'Consistency'],
      dtype='object')


In [58]:
df_master=df_master[['Circle', 'City', 'Component', 'Date', 'Hour', 'Total Customers',
       'Anomaly', 'Ratio Data', 'Ratio Voice', 'Ratio HSI', 'cluster_issue',
       'Data Volume', 'Data Volume Formula', 'Data Volume mean',
       'Data Volume std', 'Voice Affected Users',
       'Voice Affected Users Formula', 'Voice Affected Users mean',
       'Voice Affected Users std', 'HSI Affected Users',
       'HSI Affected Users Formula', 'HSI Affected Users mean',
       'HSI Affected Users std', 'num_grids', 'num_cells', 'num_sectors',
       'Indoor Frqeqency % 850', 'Indoor Frqeqency % 1800',
       'Indoor Frqeqency % 2300', 'prb_10', 'prb_10_25', 'prb_25_70',
       'prb_70_90', 'prb_90']]

In [59]:
print('Removing right Spaces from the columns')
df_master.columns = df_master.columns.str.rstrip()
df_master.columns = df_master.columns.str.lstrip()
df_master.columns = df_master.columns.str.capitalize()

Removing right Spaces from the columns


In [60]:
print('Size of the Raw Data ',df_master.shape)
print('-------------------------------')
print('-------------------------------')
print('Data types of the features in Anomaly')
print(df_master.dtypes)
print('-------------------------------')
print('-------------------------------')

Size of the Raw Data  (222556, 34)
-------------------------------
-------------------------------
Data types of the features in Anomaly
Circle                           object
City                             object
Component                         int64
Date                             object
Hour                              int64
Total customers                   int64
Anomaly                          object
Ratio data                        int64
Ratio voice                       int64
Ratio hsi                         int64
Cluster_issue                    object
Data volume                       int64
Data volume formula               int64
Data volume mean                  int64
Data volume std                   int64
Voice affected users              int64
Voice affected users formula      int64
Voice affected users mean         int64
Voice affected users std          int64
Hsi affected users                int64
Hsi affected users formula        int64
Hsi affected users mean

In [51]:
df_master=remove_whitespaces(df_master)
df_master=drop_duplicate_records(df_master)

In [52]:
df_master.columns

Index(['Circle', 'City', 'Component', 'Date', 'Hour', 'Total customers',
       'Anomaly', 'Ratio data', 'Ratio voice', 'Ratio hsi', 'Cluster_issue',
       'Data volume', 'Data volume formula', 'Data volume mean',
       'Data volume std', 'Voice affected users',
       'Voice affected users formula', 'Voice affected users mean',
       'Voice affected users std', 'Hsi affected users',
       'Hsi affected users formula', 'Hsi affected users mean',
       'Hsi affected users std', 'Num_grids', 'Num_cells', 'Num_sectors',
       'Indoor frqeqency % 850', 'Indoor frqeqency % 1800',
       'Indoor frqeqency % 2300', 'Prb_10', 'Prb_10_25', 'Prb_25_70',
       'Prb_70_90', 'Prb_90'],
      dtype='object')

In [61]:
### Basic QC's Check
print('No of Unique Circle ', df_master['Circle'].unique())
print('-----------------------------')
print('No of Cities in the circle',df_master[['Circle','City']].groupby('Circle').nunique())
print('-----------------------------')
print('No of Component in the circle',df_master[['Circle','Component']].groupby('Circle').nunique())
print('-----------------------------')
print('Unique Anomaly',df_master['Anomaly'].unique())
print('-----------------------------')
print('Unique Cluster Issue',df_master['Cluster_issue'].unique())

No of Unique Circle  ['mumbai' 'haryana' 'gujarat' 'delhi' 'punjab' 'kolkata' 'karnataka'
 'maharashtra' 'odisha' 'rajasthan' 'kerala' 'assam' 'bihar']
-----------------------------
No of Cities in the circle              City
Circle           
assam          23
bihar          51
delhi           2
gujarat        52
haryana        35
karnataka      41
kerala         33
kolkata         1
maharashtra    65
mumbai          1
odisha         28
punjab         39
rajasthan      39
-----------------------------
No of Component in the circle              Component
Circle                
assam              198
bihar              244
delhi              200
gujarat            325
haryana            121
karnataka          500
kerala             279
kolkata            208
maharashtra        351
mumbai             177
odisha             185
punjab             219
rajasthan          197
-----------------------------
Unique Anomaly [' HSI ' ' Voice ' 'Data - Voice - HSI ' 'Data ' 'Data - Voice '
 ' Voi

In [62]:
df_master.head()

,Circle,City,Component,Date,Hour,Total customers,Anomaly,Ratio data,Ratio voice,Ratio hsi,...,Num_cells,Num_sectors,Indoor frqeqency % 850,Indoor frqeqency % 1800,Indoor frqeqency % 2300,Prb_10,Prb_10_25,Prb_25_70,Prb_70_90,Prb_90
0,mumbai,mumbai,1099,2022-11-06 00:00:00,7,15211,HSI,1,2,6,...,94,27,27,25,46,20.0,48.0,20.0,0.0,2.0
1,mumbai,mumbai,1099,2022-11-06 00:00:00,6,12283,HSI,1,2,13,...,94,27,24,23,51,51.0,33.0,4.0,0.0,2.0
2,mumbai,mumbai,1090,2022-11-06 00:00:00,14,42770,HSI,2,3,4,...,100,30,20,29,50,4.0,11.0,43.0,18.0,24.0
3,mumbai,mumbai,1090,2022-11-06 00:00:00,7,25173,Voice,2,3,3,...,100,30,20,25,53,23.0,40.0,32.0,5.0,0.0
4,mumbai,mumbai,1090,2022-11-06 00:00:00,13,44454,HSI,2,3,4,...,100,30,20,29,50,3.0,14.0,43.0,18.0,22.0


In [63]:
df_master.isnull().sum()

Circle                               0
City                                 0
Component                            0
Date                                 0
Hour                                 0
Total customers                      0
Anomaly                              0
Ratio data                           0
Ratio voice                          0
Ratio hsi                            0
Cluster_issue                   109677
Data volume                          0
Data volume formula                  0
Data volume mean                     0
Data volume std                      0
Voice affected users                 0
Voice affected users formula         0
Voice affected users mean            0
Voice affected users std             0
Hsi affected users                   0
Hsi affected users formula           0
Hsi affected users mean              0
Hsi affected users std               0
Num_grids                            0
Num_cells                            0
Num_sectors              

In [64]:
### Saving in Pre Processing Folder
df_master.to_csv(pickle_path+'{}{}_Anomaly_data.csv'.format(monthname,year),index=False)
print('Data succesfully stored in:',pickle_path+'{}{}_anonaly_data.csv'.format(monthname,year))

Data succesfully stored in: /Users/raj2.gaurav/Desktop/Causal Model/02. Processed Data/01. Anomaly Data/Nov2022_anonaly_data.csv


In [32]:
path_dir:str =  "/Users/raj2.gaurav/Desktop/Causal Model/03. Collated Data/01. Anomaly Data/"
os.chdir(path_dir)
print("current dir is: %s" % (os.getcwd()))
file='Collated_Anomaly_Data.csv'

current dir is: /Users/raj2.gaurav/Desktop/Causal Model/03. Collated Data/01. Anomaly Data


In [42]:
if os.path.isfile(file):
    print("File exists")
    pickle_path_col = loc_path + 'Causal Model/03. Collated Data/01. Anomaly Data/Collated_Anomaly_Data.csv'
    print('Reading master Anomaly data for collation......')
    collated_df = pd.read_csv(pickle_path)
    print('Data read, Shape of Collated data before collation:',collated_df.shape)
    collated_df = pd.concat([collated_df, df_master])
    collated_df = collated_df.drop_duplicates()
    collated_df.reset_index(drop = True, inplace = True)
    print('Data collated, Shape of Anomaly data after collation:',collated_df.shape)
    collated_df.to_csv(pickle_path,index=False)
else:
    print("File Doesn't exist")
        #os.mkdir(folder)
    print('Creating a master Anomaly data file......')
    collated_df = pd.DataFrame()
    path = loc_path + r'/Causal Model/02. Processed Data/01. Anomaly Data/'
    pickle_path = loc_path + 'Causal Model/03. Collated Data/01. Anomaly Data/'
    collated_file_name = []
    for subdir, dirs, files in os.walk(path):
        mis_df_collated_loop = pd.DataFrame()
        for file in files:
            try:
                file_path = os.path.join(path, subdir + '/' +file)
                suffix = subdir[-9:].lower()
                collated_file_name.append(file_path)
            except:
                pass

    for file in collated_file_name:
        print(str(file))
        collated_df = pd.concat([collated_df, pd.read_csv(file)])
        print('Master Anomaly data file created, Shape of Data:',collated_df.shape)
    print(pickle_path+'Collated_Anomaly_Data.csv')
    collated_df = collated_df.drop_duplicates()
    collated_df.to_csv(pickle_path+'Collated_Anomaly_Data.csv',index=False)